## Unity Catalog Hosted Functions with Databricks Agents

### Installing Utilities and Libraries

In [ ]:
%pip install databricks-langchain==0.12.1 langchain-community==0.4.1 langchain-experimental==0.4.1 unitycatalog-ai[databricks]==0.3.2 unitycatalog-langchain[databricks]==0.3.0

### Restarting the Python Kernel

In [ ]:
dbutils.library.restartPython()

### Importing CSV File into Databricks File System (DBFS)

In [ ]:
%sh
rm -r /dbfs/spark_lab
mkdir /dbfs/spark_lab
wget -O /dbfs/spark_lab/electronics_products.csv https://raw.githubusercontent.com/kuljotSB/DatabricksGenAIEngineer/refs/heads/main/LangChain/electronics_products.csv
     

### Registering the CSV as a Delta Table in Unity Catalog

In [ ]:
df = spark.read.load('/spark_lab/electronics_products.csv', format='csv', header='true', inferSchema='true')
display(df.limit(100))

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

# Storing the CSV dataset as a Delta Table in the Databricks File System (DBFS)
deltaTablePath = '/delta/electronics_products'
df.write.format('delta').mode('overwrite').save(deltaTablePath)

# Storing the CSV dataset as a Delta Table in the Data Catalog
df.write.format('delta').saveAsTable('default.electronics_products')

### Creating the Unity Catalog Hosted Functions Client

In [ ]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

### Defining the Tool Logic and Registering in Unity Catalog

In [ ]:
%sql
CREATE OR REPLACE FUNCTION default.lookup_electronics_item(
  product_name STRING COMMENT 'Name of the product to look up. for instance if user query is "how much does green webcam cost", then product anem is "webcam" and not "green webcam"',
  product_colour STRING COMMENT 'Colour of the product to look up.'
)
RETURNS STRING
COMMENT 'Returns metadata about a specific product in the electronics_items dataset, including its ID, price, and description.'
RETURN SELECT CONCAT(
    'Product ID: ', productID, ', ',
    'Product Name: ', productName, ', ',
    'Product Colour: ', colour, ', ',
    'Price: ', price, ', '
  )
  FROM default.electronics_products
  WHERE LOWER(productName) = LOWER(product_name) 
    AND LOWER(colour) = LOWER(product_colour)
  LIMIT 1;

     

### Creating the Tool Object for usage within LangChain Agent

In [ ]:
from databricks_langchain import UCFunctionToolkit

# Create a toolkit with the Unity Catalog function
func_name = f"{CATALOG}.{SCHEMA}.lookup_electronics_item"
toolkit = UCFunctionToolkit(function_names=[func_name])

tools = toolkit.tools

### Creating the tool-calling Agent with LangChain

In [ ]:
from langchain_classic.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from databricks_langchain import (
  ChatDatabricks,
  UCFunctionToolkit,
)
import mlflow

# Initialize the LLM
LLM_ENDPOINT_NAME = "databricks-claude-haiku-4-5"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME, temperature=0.1)

# Define the prompt with agent_scratchpad placeholder
prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      "You are a helpful assistant. Make sure to use tools for additional functionality.",
    ),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
  ]
)

# Enable automatic tracing
mlflow.langchain.autolog()

# Define the agent, specifying the tools from the toolkit above
agent = create_tool_calling_agent(llm, tools, prompt)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Invoking the Agent with the Agent Executor

In [ ]:
agent_executor.invoke({"input": "Return details for the Red Graphics Card"})

In [ ]:
agent_executor.invoke({"input": "Return details for the blue keyboard"})